In [ ]:
#! pip install imageio
#! pip install PIL
#! pip install pylab
#! pip install pathlib
#! pip install scikit-learn
#! pip install d3blocks


In [ ]:
import yaml
import pandas as pd
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import cm
import seaborn as sns
import os
import imageio
from PIL import Image
from scipy import stats
from pathlib import Path
from pylab import *
from collections import Counter
import itertools
import pickle
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import cityblock
import statistics

with open("conf.yaml") as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

cov_series_path = conf["cov_series_path"]
start_date= conf["start_date"]
end_date= conf["end_date"]
out_path = conf["out_path"]
path_pois = conf["path_pois"]
scales = conf["scale_values"]
d_radius = conf["detour_radius"]
attack_path = conf["attack_path"]
path_covmap = conf["path_covmap"]
#ed_threshold = conf["euclidean_cov_threshold"]


d_radius

# Utilty function

In [ ]:

def load_centralized(scale, start, end):
    path = "../../output/coverage/scenario/mix/"+str(scale)+".0/coverages_"+start+"_"+end+".csv"
    print("cen",path)
    return pd.read_csv(path)

def load_distributed(scale, detour_radius, start, end, attack):
    if attack is True:
        path = attack_path+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"
    else:
        path = out_path+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"
    print("dis",path)
    #cov = pd.read_csv(path).drop(columns=["Unnamed: 0"])
    cov = pd.read_csv(path)
    cov = cov.join(pois_filtered.set_index('id_location'), on="id_location", how="left")
    #cov = cov.dropna()
    distributed_coverage = {'id_location':[], 'probability':[]}
    # compute the joint probability for each of the POIs
    print(len(cov.id_location.unique()))
    for poi in cov.id_location.unique():
    
        probs = cov[cov.id_location == poi]['probability']
        product = 1
        for p in probs:
            product *= (1-float(p))
        product = 1 - product    
        distributed_coverage['id_location'].append(poi)
        distributed_coverage['probability'].append(product)
        distributed_coverage['scale'] = scale
        distributed_coverage['detour_radius'] = detour_radius
        
    distributed_coverage = pd.DataFrame.from_dict(distributed_coverage).join(pois_filtered.set_index('id_location'), on="id_location", how="left")
    
    #MANFREDO save in a file dataframe with the joint probaility 
    if attack is True:
        distributed_coverage.to_csv('joint_covmap_attack.csv', index=True)
    else:
        distributed_coverage.to_csv('joint_covmap_real.csv', index=True)
    return distributed_coverage


def load_swap(scale, detour_radius, start, end, attack):
    if attack is False:
        path = out_path+"swap/no_filter/"+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"
        cov=pd.read_csv(path) ##prova
        print(cov['uid'].unique())
    else: 
        path = attack_path+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"
        #meccanismo blaclist
        path_bl = attack_path+start+"_"+end+"/blacklists_"+str(scale)+"_"+str(detour_radius)+".pk1"
        cov = exclude_attackers(path_bl,path)
        #cov=pd.read_csv(path) #to not use the blacklist mechanism
        
    print("dis",path)
    #cov = pd.read_csv(path).drop(columns=["Unnamed: 0"])
    #cov = pd.read_csv(path)  questa originale
    cov = cov.join(pois_filtered.set_index('id_location'), on="id_location", how="left")
    #cov = cov.dropna()
    distributed_coverage = {'id_location':[], 'probability':[]}
    # compute the joint probability for each of the POIs
    
    for poi in cov.id_location.unique():
    
        probs = cov[cov.id_location == poi]['probability']
        product = 1
        for p in probs:
            product *= (1-float(p))
        product = 1 - product    
        distributed_coverage['id_location'].append(poi)
        distributed_coverage['probability'].append(product)
        distributed_coverage['scale'] = scale
        distributed_coverage['detour_radius'] = detour_radius
        
    distributed_coverage = pd.DataFrame.from_dict(distributed_coverage).join(pois_filtered.set_index('id_location'), on="id_location", how="left")
    #MANFREDO save in a file dataframe with the joint probaility 
    if attack is True:
        distributed_coverage.to_csv('swap_covmap_attack.csv', index=True)
    #else:
    #    distributed_coverage.to_csv(path_covmap, index=True)
    return distributed_coverage


def plot_model(coverage, H, prefix):
    numcols, numrows = 100, 100
    dist = plt.figure(figsize=(10,10))
    #ax = dist.gca(projection='3d')
    ax=dist.add_subplot(projection='3d')

    xi = np.linspace(coverage.lon.min(), coverage.lon.max(), numrows)
    yi = np.linspace(coverage.lat.min(), coverage.lat.max(), numcols)

    #print(xi, yi)

    xi, yi = np.meshgrid(xi, yi)


    x, y, z = coverage.lon, coverage.lat, coverage.probability#/H

    # known points (long, lat)
    points = np.vstack((x,y)).T 
    # known z points
    values = z
    # all the points I want to interpolate
    wanted = (xi, yi)
    # meshgrid interpolated
    zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

    surf = ax.plot_surface(xi, yi, zi, cmap=cm.viridis,
                           linewidth=0, antialiased=False,shade=True)
    ax.set_xlabel('Longitude', fontsize=28, rotation=60, linespacing=10)
    ax.set_ylabel('Latitude', fontsize=28, rotation=60, linespacing=10)
    ax.set_zlabel('Coverage prob.', fontsize=28, rotation=60, linespacing=10)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    
    ax.xaxis.labelpad = 23
    ax.yaxis.labelpad = 23
    ax.zaxis.labelpad = 23

    
    
    ax.set_zlim(0, coverage.probability.max())#/H)
    #ax.zaxis.set_major_locator(LinearLocator(10))
    #ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    #fig.colorbar(surf, shrink=0.3, aspect=5)

    #plt.tight_layout()
    plt.savefig("images/"+prefix+"_3D.png", dpi  = 100)
    plt.show()

    fig = plt.gcf()
    fig.set_size_inches(10, 10)
    plt.contourf(xi, yi, zi, cmap = cm.viridis)
    # plt.scatter(centralized_coverage.lon, centralized_coverage.lat, c=centralized_coverage.probability, s=60, vmin=zi.min(), vmax=zi.max(),cmap=cm.Reds)
    plt.ticklabel_format(useOffset=False)
    plt.colorbar().set_label(label='Coverage prob.',size=25)
    plt.tight_layout()
    plt.xlabel("Longitude",fontsize=25)
    plt.ylabel("Latitude",fontsize=25)
    plt.savefig("images/"+prefix+"_heatmap.png", dpi  = 100, bbox_inches="tight")
    plt.show()

#utility
def euclidean_distance(covmap):
    df_final = pd.read_csv(path_covmap)
    probability_final = np.array(df_final.sort_values(by="id_location")['probability'])
    #cov_to_test = np.array(covmap.sort_index(axis=0)["probability"])
    cov_to_test = np.array(covmap.sort_values(by='id_location')["probability"])
    #cambio scala, probability*100
    #for y in range(0,1954):
    #    probability_final[y]=probability_final[y]*100
    #    cov_to_test[y] = cov_to_test[y]*100

    return np.linalg.norm(probability_final - cov_to_test)

#function to find attackers from blacklists and exclude 20% of them for the coverage map final
#input:  path blacklists and path coverage, output df of coverage_scale_dt_radius "cleaned"
def exclude_attackers(path_blacklist, path_covmap_final):
    cov = pd.read_csv(path_covmap_final)
    merged = []
    all_users=[]
    with open(path_blacklist, "rb") as fp: 
        bl = pickle.load(fp)
        #print("blacklist recovered")
        print(bl)
        all_users = list(bl.keys())
        bl_list = list(bl.values())
        merged = list(itertools.chain(*bl_list))

    list_user = np.unique(merged)
    list_user = list(list_user)
    print("USER IN BL: ",list_user)
    #retrieve means
    dict_mh = {}
    dict_ed = {}
    final_covmap = pd.read_csv(path_covmap)
    print("ALL USERS: ", all_users)
    #max_att = int(np.round(len(all_users)*20/100)) #recover the max of attackers (20%)
    max_att = 4
    print("MAX ATTACKERS: ", str(max_att))
    all_users = cov['uid'].unique()

    #eliminate node in blacklist if he sends to the server a blank coverage map
    for node in list_user:
        if node not in all_users:
            print("NODO: ",node)
            list_user.remove(node)
    for user in all_users:
        df_cov_user = cov[cov['uid']==user]
        probability_final = np.array(final_covmap.sort_values(by="id_location")['probability'])
        cov_to_test = np.array(df_cov_user.sort_values(by='id_location')["probability"])
        dict_ed[int(user)] = np.linalg.norm(probability_final - cov_to_test)
        dict_mh[int(user)] = cityblock(probability_final, cov_to_test)

    #media delle manhattan e euclidean distance tra tutte le mappe che il server riceve
    ed_mean = mean(list(dict_ed.values()))
    mh_mean = mean(list(dict_mh.values()))
    print("ed_mean: ",ed_mean)
    print("mh_mean: ",mh_mean)

    #sorted by value (distance)
    sorted_dict_mh = sorted(dict_mh.items(), key=lambda x:x[1], reverse=True)
    sorted_dict_ed = sorted(dict_ed.items(), key=lambda x:x[1], reverse=True)

    #check 
    counter = 0
    for user in sorted_dict_mh:
        if user[0] in list_user:
            df_cov_user = cov[cov['uid']==user[0]]
            print("USER : {} EUCLIDEAN DISTANCE: {}",user[0],dict_ed[user[0]])
            print("USER : {} MANHATTAN DISTANCE: {}",user[0],dict_mh[user[0]])
            
            if ((dict_ed[user[0]] > ed_mean)            #mh_std/2 e ed_std/2
                and (dict_mh[user[0]] > mh_mean)) :
                print("elimino cov map di: ", user[0])
                #remove coverage map of user from cov
                for index, row in cov.iterrows():
                     if row.uid == user[0]:
                         cov = cov.drop(index)
                counter += 1
                if counter == max_att:
                    break
    return cov

In [ ]:
#cella per salvare la giusta coverage map finale senza attacco
#start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
start = "2009-05-16"
end_date = ["2009-05-23 00:00:00"]
pois_filtered = pd.read_csv(path_pois) 
for end in end_date:
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    swap_real = load_swap(100,800,start,end,False)

In [ ]:
#mettere a confronto distanze delle coverage degli utenti nei 3 periodi
scale = 100
detour_radius = 800
attackers = [30,128,84,140]
pois_filtered = pd.read_csv(path_pois)  
df_1 = pd.read_csv(attack_path+"2009-05-08_2009-05-15/coverage_100_800.csv") #recover period 1
df_2 = pd.read_csv(attack_path+"2009-05-16_2009-05-23/coverage_100_800.csv") #recover period 2
df_3 = pd.read_csv(attack_path+"2009-05-24_2009-05-31/coverage_100_800.csv") #recover period 3
df_4 = pd.read_csv(attack_path+"2009-05-08_2009-05-31/coverage_100_800.csv") #recover period 4
df_list = [df_1,df_2,df_3,df_4]

counter = 1
for df in df_list:
    print("Period: ", counter)
    list_user = df['uid'].unique()
    print(list_user)
    dict_ed = {}
    dict_mh = {}
    dicts = []
    manhattan_threshold = 0
    for user in list_user:
        df_cov_user = df[df['uid']==user]
        if counter == 1 or counter == 4:
            final_covmap = pd.read_csv("swap_covmap_real.csv")
        elif counter == 2:
            final_covmap = pd.read_csv("swap_covmap_real2.csv")
        else:
            final_covmap = pd.read_csv("swap_covmap_real3.csv")
        probability_final = np.array(final_covmap.sort_values(by="id_location")['probability'])
        cov_to_test = np.array(df_cov_user.sort_values(by='id_location')["probability"])
        dict_ed[int(user)] = np.linalg.norm(probability_final - cov_to_test)
        dict_mh[int(user)] = cityblock(probability_final, cov_to_test)
    print("Mean Euclidean Distance:", mean(list(dict_ed.values())))
    print("Mean Manhattan Distance:", mean(list(dict_mh.values())))
    #print("Mean difference manhattan from manhattan mean:", mean(np.absolute(np.array(list(dict_mh.values()))-mean(list(dict_mh.values())))))
    print("STANDARD DEVIATION MANHATTAN: ",np.std(list(dict_mh.values())))
    #print("Mean difference euclidean from euclidean mean:", mean(np.absolute(np.array(list(dict_ed.values()))-mean(list(dict_ed.values())))))
    print("STANDARD DEVIATION EUCLIDEAN: ",np.std(list(dict_ed.values())))
    dicts.append(dict_ed)
    dicts.append(dict_mh)
    inner_counter = 1
    for dictx in dicts:
        #ordino dictionary per id utente
        keys = list(dictx.keys())
        keys.sort()
        colors = []
        i = 0
        for key in keys:
            if key in attackers:
                colors.append("red")
            else:
                colors.append("blue")
            i +=1
            
        sorted_dict = {i: dictx[i] for i in keys}
        bins = range(len(sorted_dict.keys()))
        plt.scatter(bins, sorted_dict.values(), c=colors)
        plt.xlabel('Users', fontsize=16)
        
        plt.xticks(bins, sorted_dict.keys())
        #for manhattan
        if inner_counter == 2:
            horizontal_line_sup = mean(list(dict_mh.values())) #+ np.std(list(dict_mh.values()))/2#mean(np.absolute(np.array(list(dict_mh.values()))-mean(list(dict_mh.values()))))  
            #horizontal_line_inf = mean(list(dict_mh.values())) - mean(np.absolute(np.array(list(dict_mh.values()))-mean(list(dict_mh.values()))))
            plt.ylabel('Manhattan Distance', fontsize=16)
        #for euclidean
        if inner_counter == 1:
            horizontal_line_sup = mean(list(dict_ed.values())) #+ np.std(list(dict_ed.values()))/2 #mean(np.absolute(np.array(list(dict_ed.values()))-mean(list(dict_ed.values()))))  
            #horizontal_line_inf = mean(list(dict_ed.values())) - mean(np.absolute(np.array(list(dict_ed.values()))-mean(list(dict_ed.values()))))
            plt.ylabel('Euclidean Distance', fontsize=16)
        plt.axhline(y=horizontal_line_sup, color='r', linestyle='--', label='Horizontal Line Sup')
        #plt.axhline(y=horizontal_line_inf, color='r', linestyle='--', label='Horizontal Line Inf')
        plt.show()
        inner_counter += 1
    #update counter
    counter += 1    


In [ ]:
#generazione matrice dei contatti

import numpy as np
import matplotlib.pyplot as plt

from d3blocks import D3Blocks  #for chord graph

start = "2009-05-08"
end = "2009-05-31"

# Example data: Replace this with your actual data
contacts = pd.read_csv(attack_path+start+"_"+end+"/contacts_100_800.csv")

#####################chord graph###############
source = contacts['node_a']
target = contacts['node_b']
weights = contacts['contacts']
df = pd.DataFrame(data=np.c_[source, target, weights], columns=['source','target','weight'])

# Initialize
d3 = D3Blocks(frame=False)
d3.chord(df, color='source', opacity='source', cmap='Set2')

#################################################
all_users_a = contacts['node_a'].unique()
all_users_b = contacts['node_b'].unique()
all_users = np.union1d(all_users_a, all_users_b)
#all_users = all_users.unique()
print("ALL USERS: ", all_users)
print("NUM USERS: ", len(all_users))
num_users = len(all_users)
contact_data = []
for user_a in all_users:
    user_a_contacts = []
    for user_b in all_users:
        if user_b != user_a:
            if len(contacts.loc[contacts.node_a==user_a][contacts.node_b==user_b].values)!=0:
                #print(contacts.loc[contacts.node_a==user_a][contacts.node_b==user_b].values[0][3])
                user_a_contacts.append(contacts.loc[contacts.node_a==user_a][contacts.node_b==user_b].values[0][3])
            else:
                #print(contacts.loc[contacts.node_a==user_b][contacts.node_b==user_a].values[0][3])
                user_a_contacts.append(contacts.loc[contacts.node_a==user_b][contacts.node_b==user_a].values[0][3])
        else:
            user_a_contacts.append(0)
    contact_data.append(user_a_contacts)
            
print(contact_data)
# Create the contact matrix
contact_matrix = np.array(contact_data)
print(contact_matrix)
# Plotting the matrix with cell annotations
plt.figure(figsize=(18, 16))
plt.matshow(contact_matrix, fignum=1, cmap='viridis')  #DECIDERE BENE COLORE

plt.xticks(np.arange(num_users))
plt.yticks(np.arange(num_users))
plt.gca().set_xticklabels(all_users)  # Adjust if users are numbered differently
plt.gca().set_yticklabels(all_users)  # Adjust if users are numbered differently

#fill the matrix
for i in range(contact_matrix.shape[0]):
    for j in range(contact_matrix.shape[1]):
        plt.text(j, i, str(contact_matrix[i, j]), va='center', ha='center', color='red')

cbar = plt.colorbar()
cbar.set_label(label='Number of contacts', size=15)
plt.title('Contact Matrix', fontsize=20)
plt.xlabel('Users', fontsize=18)
plt.ylabel('Users', fontsize=18, rotation=90)
#plt.savefig("08-31_contact_matrix_augmented.png", dpi  = 100)
plt.show()


In [ ]:
#differenze principali POI 
start = "2009-05-08"
end = "2009-05-31"
scale = 100
detour_radius = 800
pois_filtered = pd.read_csv(path_pois)  
H = len(pois_filtered)

df_real = load_swap(scale,detour_radius,start,end,False)
df_attack = load_swap(scale,detour_radius,start,end,True)

df_real = df_real.sort_values(by="id_location")
df_attack = df_attack.sort_values(by="id_location")
#probability_final = np.array(df_real.sort_values(by="id_location")['probability'])
#probability_attack = np.array(df_attack.sort_values(by='id_location')['probability'])

counter = 0
POI_real = {}
POI_attack = {}
for index,row in df_real.iterrows():
    #print(df_attack.loc[df_attack.id_location==row.id_location,'probability'].values[0])
    prob_attack = df_attack.loc[df_attack.id_location==row.id_location,'probability'].values[0]
    if np.absolute(row.probability-prob_attack)>=0.5:
        POI_real[row.id_location] = row.probability
        POI_attack[row.id_location] = prob_attack
        counter+=1

print("numero POI variati molto: ",counter)
keys = POI_real.keys()  # Assuming both dictionaries have the same keys
print("keys: ",keys)
print("len keys: ", len(keys))
values_real = POI_real.values()
values_attack = POI_attack.values()

x2 = np.arange(len(keys))

plt.scatter(x2, values_real, label='No Attack')
plt.scatter(x2, values_attack, label='Attack')

plt.xlabel('ID location', fontsize=18)
plt.ylabel('Values', fontsize=18)
#plt.xticks(np.arange(len(keys)),keys, rotation=90)
#plt.xticks(np.arange(0,len(keys),step=1), keys, rotation=90)
#plt.tick_params(axis='x', which='major', labelsize=3)
plt.xticks(x2, keys, rotation=90)

plt.title('POI most changed')
plt.legend()
plt.figure(figsize=(18, 16))
plt.show()
#print("POI_REAL: ", POI_real)
#print("POI_ATTACK: ", POI_attack)

In [ ]:
#generate coverage map final by server with blacklist mechanism

#start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
start = "2009-05-08"
end_date = ["2009-05-31 00:00:00"]
scale = 100
detour_radius = 800
pois_filtered = pd.read_csv(path_pois)  
H = len(pois_filtered)

#attackers= [128,30]
#attackers=[30]
for end in end_date:
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    #distributed_coverage = load_distributed(scale,detour_radius,start,end,False)
    df_real = load_swap(scale,detour_radius,start,end,False) #sono swap senza attacco
    swap_attack = load_swap(scale,detour_radius,start,end,True)
    #df_distributed = pd.read_csv("joint_covmap_real.csv")
    df_swap = pd.read_csv("swap_covmap_attack.csv")
    #df_real = pd.read_csv("swap_covmap_real.csv")

    
    real = df_real['probability']
    attack = df_swap['probability']
    bins = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    #x = plt.hist(real, bins, label=['real'])
    #y = plt.hist(attack, bins, label=['attack'])
    plt.hist([real,attack], bins, label=['real','attack'])
    plt.legend(loc='upper center')
    plt.xlabel('Probability',fontsize=14)
    plt.ylabel('Number of POI',fontsize=14)
    plt.title("Period "+start+"/"+end, fontsize=14)
    plt.xticks(bins)
    plt.show()

    
    probability_final = np.array(df_real.sort_values(by="id_location")['probability'])
    probability_attack = np.array(df_swap.sort_values(by='id_location')['probability'])
    print("FINAL EUCLIDEAN DISTANCE: ",np.linalg.norm(probability_final - probability_attack) )
    print("FINAL MANHATTAN DISTANCE: ", cityblock(probability_final,probability_attack))
    df_real = df_real.dropna()
    df_swap = df_swap.dropna()
    plot_model(df_real,H, "real"+str(scale)+"_"+str(detour_radius)+"_"+str(start)+"_"+str(end))
    plot_model(df_swap, H,  "attack"+str(scale)+"_"+str(detour_radius)+"_"+str(start)+"_"+str(end))

    #plot the heatmap of the difference 
    print("PLOT DIFFERENCE:")
    df_difference = df_real.sort_values(by="id_location")
    df_difference['probability'] = df_swap.sort_values(by='id_location')['probability'] - df_difference['probability'] 
    plot_model(df_difference, H, "difference"+str(scale)+"_"+str(detour_radius)+"_"+str(start)+"_"+str(end))
    
    print(stats.ks_2samp(df_real.sort_values(by="id_location")['probability'], df_swap.sort_values(by="id_location")['probability']))

In [ ]:
#grafico per paragonare risultati con o senza filter e blacklist

#euclidean
list_blacklist = [5.18, 5.31, 4.52,4.73]
list_no_blacklist = [16.15, 15.38, 4.55,11.23]
list_no_defense = [20.12, 24.98, 17.11,20.25]

#manhattan
#list_blacklist = [86.1, 84.77, 54.32,73.54]
#list_no_blacklist = [342.76, 312.48, 53.07,207.79]
#list_no_defense = [529.76, 752.67, 393,581.93]

x_axis = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4"]
x_positions = np.arange(len(x_axis))

plt.bar(x_positions - 0.3, list_no_defense, width=0.3, label='No Defense')
plt.bar(x_positions - 0.00, list_no_blacklist, width=0.3, label='Filter')
plt.bar(x_positions + 0.3, list_blacklist, width=0.3, label='Filter & Blacklist')

plt.xlabel('Time Period', fontsize=16)
plt.ylabel('Euclidean Distance', fontsize=16)
plt.title('Filter & Blacklist results')
plt.legend()

# Setting x-axis ticks and labels
plt.xticks(x_positions, x_axis, rotation=45)

# Display the plot
plt.show()

In [ ]:
#graph with final results for all scenarios with both attacks and both Malicious worker ratio
#lists no defense
random_ed_10_nodefense = [36.72,37.30,36.26,30.12]
random_ed_20_nodefense = [36.73,37.33,36.48,30.12]
random_mh_10_nodefense = [1507.69,1527.19,1470.70,1094.53]
random_mh_20_nodefense = [1508.03,1528.56,1480.21,1094.53]
PASL_ed_10_nodefense = [20.12,24.98,17.11,20.26]
PASL_ed_20_nodefense = [19.87,19.45,14.23,20.95]
PASL_mh_10_nodefense = [529.76,752.67,393,581.93]
PASL_mh_20_nodefense = [526.68,498.75,312.59,615.55]

#lists defense
random_ed_10 = [3.33,3.54,3.94,4.47]
random_ed_20 = [5.72,5.7,5.85,5.44]
random_mh_10 = [45.38,42.36,46.17,64.32]
random_mh_20 = [91.1,92.27,97.37,103.1]
PASL_ed_10 = [5.18,5.31,4.52,4.73]
PASL_ed_20 = [11.1,10.1,8.7,13.3]
PASL_mh_10 = [86.1,84.77,54.32,73.54]
PASL_mh_20 = [243,214.8,170,362.1]

x_axis = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4"]
x_positions = np.arange(len(x_axis))

plt.plot(x_axis, random_ed_10_nodefense, label='Random_10_noDef', marker='o', ls="--")
plt.plot(x_axis, random_ed_20_nodefense, label='Random_20_noDef', marker='o', ls="--")
plt.plot(x_axis, PASL_ed_10_nodefense, label='PASL_10_noDef', marker='o', ls="--")
plt.plot(x_axis, PASL_ed_20_nodefense, label='PASL_20_noDef', marker='o', ls="--")
plt.plot(x_axis, random_ed_10, label='Random_10', marker='o')
plt.plot(x_axis, random_ed_20, label='Random_20', marker='o')
plt.plot(x_axis, PASL_ed_10, label='PASL_10', marker='o')
plt.plot(x_axis, PASL_ed_20, label='PASL_20', marker='o')

plt.xlabel('Scenario', fontsize=18)
plt.ylabel('Euclidean Dist.', fontsize=18)
plt.title("Final Result Euclidean Distance", fontsize=18)
plt.legend(loc=(1,0.5))
plt.show()

plt.plot(x_axis, random_mh_10_nodefense, label='Random_10_noDef', marker='o', ls="--")
plt.plot(x_axis, random_mh_20_nodefense, label='Random_20_noDef', marker='o', ls="--")
plt.plot(x_axis, PASL_mh_10_nodefense, label='PASL_10_noDef', marker='o', ls="--")
plt.plot(x_axis, PASL_mh_20_nodefense, label='PASL_20_noDef', marker='o', ls="--")
plt.plot(x_axis, random_mh_10, label='Random_10', marker='o')
plt.plot(x_axis, random_mh_20, label='Random_20', marker='o')
plt.plot(x_axis, PASL_mh_10, label='PASL_10', marker='o')
plt.plot(x_axis, PASL_mh_20, label='PASL_20', marker='o')

plt.xlabel('Scenario', fontsize=18)
plt.ylabel('Manhattan Dist.', fontsize=18)
plt.title("Final Result Manhattan Distance", fontsize=18)
plt.legend(loc=(1,0.50))
plt.show()

## KS test centralized vs distributed
- scale = 50
- detour_radius = 800

In [ ]:
scale = 100 #50
detour_radius = 800
pois_filtered = pd.read_csv(path_pois)  
H = len(pois_filtered)
start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")

end_date = ["2009-05-15 00:00:00"]
for end in end_date:
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    centralized_coverage = load_centralized(scale, start, end)
    distributed_coverage = load_distributed(scale,detour_radius,start,end, False)
    print(stats.ks_2samp(distributed_coverage.sort_values(by="id_location")['probability'], centralized_coverage.join(distributed_coverage.set_index('id_location'), on="id_location", how="right", rsuffix="_distr").sort_values(by="id_location")['probability']))

## KS test distributed vs Swap
- 1 coverage map obtained as joint probability
- KS distribution for every user
- contacts

In [ ]:
# KS using 1 coverage map given by the joint probability
scale = 800
detour_radius = 100
pois_filtered = pd.read_csv(path_pois)  
H = len(pois_filtered)
start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
print(start)
end_date = ["2009-05-25 00:00:00","2009-05-31 00:00:00","2009-06-05 00:00:00"]
#end_date = ["2009-05-07 00:00:00", "2009-05-15 00:00:00"]
for end in end_date:
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    swap_coverage = load_swap(scale,detour_radius,start,end,False)
    distributed_coverage = load_distributed(scale,detour_radius,start,end,False)
    print(stats.ks_2samp(distributed_coverage.sort_values(by="id_location")['probability'], swap_coverage.join(distributed_coverage.set_index('id_location'), on="id_location", how="right", rsuffix="_distr").sort_values(by="id_location")['probability']))

In [ ]:
# KS(swap, distributed) distribution for every user
#end_dates = ["2009-05-12 00:00:00","2009-05-14 00:00:00","2009-05-16 00:00:00","2009-05-17 00:00:00","2009-05-20 00:00:00","2009-05-25 00:00:00","2009-05-31 00:00:00","2009-06-05 00:00:00"]
end_dates = ["2009-05-25 00:00:00","2009-05-31 00:00:00","2009-06-05 00:00:00"]
sns.set(style="ticks",font_scale=2.6)

fig = plt.figure(figsize=(10,8))
scale = 800
detour_radius = 100
ks_df = pd.DataFrame(columns=["statistic","p-value","uid","scenario"])
start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
for end in end_dates:
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    distributed_path = out_path+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"
    swap_path = out_path+"swap/"+start+"_"+end+"/coverage_"+str(scale)+"_"+str(detour_radius)+".csv"

    distributed_df = pd.read_csv(distributed_path)
    swap_df = pd.read_csv(swap_path)
    for user in distributed_df.uid.unique():
        #print(user)
        user_cov_dis = distributed_df[distributed_df.uid == user]
        user_swap_dis = swap_df[swap_df.uid == user]
        if user_swap_dis.empty:
            continue
        user_ks = stats.ks_2samp(user_cov_dis.sort_values(by="id_location")['probability'], user_swap_dis.join(user_cov_dis.set_index('id_location'), on="id_location", how="right", rsuffix="_distr").sort_values(by="id_location")['probability'])
        ks_df = ks_df.append({"statistic":user_ks[0],"p-value": user_ks[1] ,"uid":user,"scenario": end},ignore_index=True)


#print(ks_df)

# sns.jointplot(data=ks_df, x = "KS", y="p-value", hue="scenario",height=10)
# plt.ylim([0,1.2])


# pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
# g = sns.FacetGrid(ks_df, row="scenario", hue="scenario", palette = pal,despine = True, aspect = 4,sharey=False,row_order=["2009-05-12","2009-05-14","2009-05-31"])
# g.map(sns.kdeplot, "p-value",bw_adjust=.5, clip_on=True,fill=True, alpha=1, linewidth=1.5)
# g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
# g.set_xlabels('p-value')
# g.set_yticklabels([])
# g.despine(bottom=True, left=True)
#######

ks_df = pd.melt(ks_df,id_vars=['scenario'],var_name='metric', value_vars=["statistic","p-value"],value_name='values')
#g = sns.FacetGrid(ks_df,hue="scenario", col="metric" , col_wrap=2, sharex=False,sharey=False,despine = True, palette="tab10",col_order=["p-value","statistic"],hue_order=["2009-05-25","2009-05-31","2009-06-05"],aspect=3,height=5)
#g.map(sns.kdeplot, "values",fill=True,cbar=True)

#g = sns.catplot(data=ks_df,col="metric",x="scenario",y="values",kind="box",aspect=2,sharey=False,col_order=["p-value","statistic"])
g = sns.catplot(data=ks_df,col="metric",x="scenario",y="values",kind="box",aspect=2,sharey=False,col_order=["p-value","statistic"], palette="tab10")

#g.add_legend(loc="upper right")
g.set_xlabels("")
g.set_xticklabels(["18 days","24 days","29 days"])
ax = g.axes


#print(ax)
#ax[0].legend(loc='upper center', bbox_to_anchor=(1, -0.20),fancybox=True, shadow=True, ncol=5, labels=["18 days","24 days", "29 days"])

plt.savefig("images/KS-user-distribution.png", dpi  = 100)



In [ ]:
# Contacts
end_dates = ["2009-05-25 00:00:00", "2009-06-05 00:00:00"]
sns.set(style="ticks",font_scale=3)
scale = 800
detour_radius = 100
ks_df = pd.DataFrame(columns=["KS","p-value","uid","scenario"])
start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
for end in end_dates:
    fig = plt.figure(figsize=(12,12))
    end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    contacts_path = out_path+"swap/"+start+"_"+end+"/contacts_"+str(scale)+"_"+str(detour_radius)+".csv"
    contacts = pd.read_csv(contacts_path).pivot("node_a", "node_b", "contacts")
    contacts=contacts.fillna(0)
    print(contacts.sum().sum())
    g = sns.heatmap(data=contacts,linewidth=.5,cmap="crest",square=True, vmin=0,vmax=700,cbar_kws={"shrink": .82})
    plt.xlabel(r'node ID')
    plt.ylabel(r'node ID')
    plt.tight_layout()
    plt.savefig("images/user-contacts_"+str(end)+".png", dpi  = 100)


## Analysis of the Distributed Model

In [ ]:
sns.set(font_scale=2.2)  
sns.set_style("ticks")
plt.figure(figsize=(10, 10))

pois_filtered = pd.read_csv(path_pois)  

start = "2009-05-01"
end = "2009-05-15"
scales = [50,100,500,800]
d_radius = [100,200,500,800]
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)

dist_coveraged = pd.DataFrame()
for scale in scales:
    for detour_radius in d_radius:
        d = load_distributed(scale, detour_radius, start, end, False)
        dist_coveraged = pd.concat([dist_coveraged,d])

        
g = sns.FacetGrid(dist_coveraged, row="scale", col="detour_radius",despine = True, aspect = 2)
g.map(sns.kdeplot, "probability",bw_adjust=.5, clip_on=True,fill=True,color = "green", alpha=0.6, linewidth=1.5)


# # passing color=None to refline() uses the hue mapping
#g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)
g.set_xlabels('$P(C^h)$')
g.savefig('./images/distributed_analysis.png')

## Plot Models

In [ ]:
pois_filtered = pd.read_csv(path_pois)  
H = len(pois_filtered)
start = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")

scales = [50]

for scale in scales:
    for end in end_date:
        end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
        centralized_coverage = load_centralized(scale, start, end)
        plot_model(centralized_coverage,H, "centralized_"+str(scale)+"_"+str(end))
        
for scale in scales:
    for end in end_date:
        end =  datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
        for detour_radius in d_radius:
            distributed_coverage = load_distributed(scale,detour_radius,start,end,False)
            plot_model(distributed_coverage,H, "distributed_"+str(scale)+"_"+str(detour_radius)+"_"+str(end))



# Load the coverage output

In [ ]:
series_dir = run_path+"cov_series/"
series_files = os.listdir(series_dir)

#geo_filtered = pd.read_csv('../../../datasets/mobility/Geolife_Trajectories_1.3/Data/geolife_full_augmented.csv')
pois_filtered = pd.read_csv(path_pois)  
cov = pd.read_csv(run_path+"coverage_"+str(scale)+"_"+str(detour_radius)+".csv").drop(columns=["Unnamed: 0"])
image_dst = run_path+"images/"
Path(image_dst).mkdir(parents=True, exist_ok=True)


cov_series = []
if '.ipynb_checkpoints' in series_files:
    series_files.remove('.ipynb_checkpoints')
#print(series_files)

for file in series_files:
    #print(series_dir + file)
    df = pd.read_csv(series_dir + file).rename(columns={"Unnamed: 0": "id_location"})
    node = file.split("_")[0]
    df['node'] = node
    cov_series.append(df)
    
cov_series = pd.concat(cov_series)
#harmonize dtype
#cov_series[['id_location']] = cov_series[['id_location']].astype(int)
#pois_filtered[['id_location']] = pois_filtered[['id_location']].astype(int)
#df1 = cov_series[cov_series.isna().any(axis=1)]
#print("df1",df1)
cov_series = cov_series.join(pois_filtered.set_index('id_location'), on="id_location", how="left")
cov_series = cov_series.dropna()  #added Alex 17/10
#df2 = cov_series[cov_series.isna().any(axis=1)]
#print("df2",df2)
#print('{5.2f}'.format(df2.id_location))

cov = cov.join(pois_filtered.set_index('id_location'), on="id_location", how="left")
cov = cov.dropna()

locations = pd.read_csv(path_pois)
H = len(locations)
print("Locations found ", H)

###### Load the distributed coverage
distributed_coverage = {'id_location':[], 'probability':[]}
# compute the joint probability for each of the POIs

for poi in cov.id_location.unique():
    
    probs = cov[cov.id_location == poi]['probability']
    product = 1
    
    for p in probs:
        product *= (1-float(p))
    product = 1 - product    
    distributed_coverage['id_location'].append(poi)
    distributed_coverage['probability'].append(product)

    
distributed_coverage = pd.DataFrame.from_dict(distributed_coverage).join(pois_filtered.set_index('id_location'), on="id_location", how="left")


##### Load the centralized coverage
centralized_coverage = pd.read_csv(centralized_model_output)


### Coverage series examination

we are going to observe the variation during the time of a chosen node the coverage map

In [ ]:
node = '1'
node_serie = cov_series[cov_series.node == node]

numcols, numrows = 100, 100
print(image_dst)
Path(f"{image_dst}/series/{node}/3D/").mkdir(parents=True, exist_ok=True)
Path(f"{image_dst}/series/{node}/heatmap/").mkdir(parents=True, exist_ok=True)



count = 1 
for c in node_serie.copy_id.unique():
    data = node_serie[node_serie.copy_id == c]
    
    fig = plt.figure(figsize=(15,15))
    ax = fig.gca(projection='3d')
    
    xi = np.linspace(data.lon.min(), data.lon.max(), numrows)
    yi = np.linspace(data.lat.min(), data.lat.max(), numcols)

    #print(xi, yi)

    xi, yi = np.meshgrid(xi, yi)


    x, y, z = data.lon, data.lat, data.probability


    # known points (long, lat)
    points = np.vstack((x,y)).T 
    # known z points
    values = z
    # all the points I want to interpolate
    wanted = (xi, yi)
    # meshgrid interpolated
    zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

    surf = ax.plot_surface(xi, yi, zi, cmap=cm.Reds,
                           linewidth=0, antialiased=False,shade=True)

    ax.set_zlim(0, 1)
    #ax.zaxis.set_major_locator(LinearLocator(10))
    #ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    #fig.colorbar(surf, shrink=0.3, aspect=5)
    
    bid = node_serie[node_serie.copy_id == c].bid.unique()[0]
    print(bid)
    #plt.show()
    plt.ioff()
    fig = plt.gcf()
    fig.set_size_inches(7, 7)
    plt.title(f'{bid}')
    plt.tight_layout()
    plt.savefig(f'{img_dest}/series/{node}/3D/{count}_{node}_3D_grid.png', dpi  = 100)
    plt.clf()
    plt.close()
    
    fig = plt.gcf()
    fig.set_size_inches(7, 7)
    plt.contourf(xi, yi, zi)
    plt.scatter(data.lon, data.lat, c=data.probability, s=60, vmin=zi.min(), vmax=zi.max(),cmap=cm.Reds)
    plt.ticklabel_format(useOffset=False)
    plt.colorbar()
    plt.title(f'{bid}')
    plt.tight_layout()
    plt.savefig(f"{img_dest}/series/{node}/heatmap/{count}_{node}_heatmap.png", dpi  = 300)
    
    plt.close()
    count += 1


In [ ]:
#images = [imageio.imread(f"{img_dest}/series/{node}/heatmap/{i + 1}_{node}_heatmap.png") for i in range(len(os.listdir(f'{img_dest}/series/{node}/heatmap/')) - 2)]
#kwrgs = { 'duration': 0.5 }
#imageio.mimsave(f'{img_dest}/{node}_heatmap.gif', images, **kwrgs)
#images = [imageio.imread(f"{img_dest}/series/{node}/3D/{i + 1}_{node}_3D_grid.png") for i in range(len(os.listdir(f'{img_dest}/series/{node}/3D/')) - 2)]
#kwrgs = { 'duration': 0.5 }
#imageio.mimsave(f'{img_dest}/{node}_3D.gif', images, **kwrgs)
node = '128'
writer = imageio.get_writer(f"{img_dest}/{node}_heatmap.mp4", fps=3)
for i in range(len(os.listdir(f'{img_dest}/series/{node}/heatmap/')) - 2):
    im = imageio.imread(f"{img_dest}/series/{node}/heatmap/{i + 1}_{node}_heatmap.png")
    writer.append_data(im)
writer.close()

writer = imageio.get_writer(f"{img_dest}/{node}_3D_grid.mp4", fps=3)
for i in range(len(os.listdir(f'{img_dest}/series/{node}/3D/')) - 2):
    im = imageio.imread(f"{img_dest}/series/{node}/3D/{i + 1}_{node}_3D_grid.png")
    writer.append_data(im)
writer.close()

#### Time-lapse of the coverage map
![Gif](images/exp_2/128.gif "node 17")

### Final coverages examination

plot a 3d graphic for each of the nodes

In [ ]:
coverages = []
fused_coveragemcols, numrows = 100, 100

coverages = []

for n in cov.uid.unique():
    data = cov[cov.uid == n]
    
    fig = plt.figure(figsize=(15,15))
    ax = fig.gca(projection='3d')
    
    xi = np.linspace(data.lon.min(), data.lon.max(), numrows)
    yi = np.linspace(data.lat.min(), data.lat.max(), numcols)

    #print(xi, yi)

    xi, yi = np.meshgrid(xi, yi)


    x, y, z = data.lon, data.lat, data.probability


    # known points (long, lat)
    points = np.vstack((x,y)).T 
    # known z points
    values = z
    # all the points I want to interpolate
    wanted = (xi, yi)
    # meshgrid interpolated
    zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

    surf = ax.plot_surface(xi, yi, zi, cmap=cm.Reds,
                           linewidth=0, antialiased=False,shade=True)

    ax.set_zlim(0, 1)
    #ax.zaxis.set_major_locator(LinearLocator(10))
    #ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    #fig.colorbar(surf, shrink=0.3, aspect=5)
    
    plt.tight_layout()
    fig = plt.gcf()
    fig.set_size_inches(7, 7)
    plt.title(f'Coverage map of node {n}')
    Path(f"{img_dest}/nodes/").mkdir(parents=True, exist_ok=True)
    plt.savefig(f'{img_dest}/nodes/{n}_final_3D_grid.png', dpi  = 100)
    #plt.show()
    plt.close()

### Distributed vs Centralized Coverage

Distributed coverage maps fusion

In [ ]:

numcols, numrows = 100, 100

dist = plt.figure(figsize=(15,15))
ax = dist.gca(projection='3d')

xi = np.linspace(distributed_coverage.lon.min(), distributed_coverage.lon.max(), numrows)
yi = np.linspace(distributed_coverage.lat.min(), distributed_coverage.lat.max(), numcols)

#print(xi, yi)

xi, yi = np.meshgrid(xi, yi)


x, y, z = distributed_coverage.lon, distributed_coverage.lat, distributed_coverage.probability/H


# known points (long, lat)
points = np.vstack((x,y)).T 
# known z points
values = z
# all the points I want to interpolate
wanted = (xi, yi)
# meshgrid interpolated
zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

surf = ax.plot_surface(xi, yi, zi, cmap=cm.viridis,
                       linewidth=0, antialiased=False,shade=True)
ax.set_xlabel(r'Longitude', fontsize=15, rotation=60)
ax.set_ylabel('Latitude', fontsize=15, rotation=60)
ax.set_zlabel('Density', fontsize=15, rotation=60)


ax.set_zlim(0, distributed_coverage.probability.max()/H)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.3, aspect=5)

plt.tight_layout()
#plt.savefig(image_dst+"/distributed_final_3D_grid.png", dpi  = 150)
plt.show()

fig = plt.gcf()
fig.set_size_inches(10, 7)
plt.contourf(xi, yi, zi)
#plt.scatter(distributed_coverage.lon, distributed_coverage.lat, c=distributed_coverage.probability, s=60, vmin=zi.min(), vmax=zi.max(),cmap=cm.Reds)
plt.ticklabel_format(useOffset=False)
plt.colorbar()
plt.tight_layout()
plt.xlabel("Longitude")
plt.ylabel("Latitude")

#plt.savefig(image_dst+"/distributed_final_heatmap.png", dpi  = 150)
plt.show()

In [ ]:
#centralized = centralized[centralized.lon <= 116.411544]
numcols, numrows = 100, 100


dist = plt.figure(figsize=(15,15))
ax = dist.gca(projection='3d')

xi = np.linspace(centralized_coverage.lon.min(), centralized_coverage.lon.max(), numrows)
yi = np.linspace(centralized_coverage.lat.min(), centralized_coverage.lat.max(), numcols)

#print(xi, yi)

xi, yi = np.meshgrid(xi, yi)


x, y, z = centralized_coverage.lon, centralized_coverage.lat, centralized_coverage.probability/H

# known points (long, lat)
points = np.vstack((x,y)).T 
# known z points
values = z
# all the points I want to interpolate
wanted = (xi, yi)
# meshgrid interpolated
zi = griddata(points, values, wanted,method="linear", fill_value = 0) 

surf = ax.plot_surface(xi, yi, zi, cmap=cm.viridis,
                       linewidth=0, antialiased=False,shade=True)
ax.set_xlabel(r'Longitude', fontsize=15, rotation=60)
ax.set_ylabel('Latitude', fontsize=15, rotation=60)
ax.set_zlabel('Density', fontsize=15, rotation=60)

ax.set_zlim(0, distributed_coverage.probability.max()/H)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.3, aspect=5)

plt.tight_layout()
#plt.savefig(image_dst+"/centralized_final_3D_grid.png", dpi  = 150)
plt.show()

fig = plt.gcf()
fig.set_size_inches(10, 7)
plt.contourf(xi, yi, zi)
# plt.scatter(centralized_coverage.lon, centralized_coverage.lat, c=centralized_coverage.probability, s=60, vmin=zi.min(), vmax=zi.max(),cmap=cm.Reds)
plt.ticklabel_format(useOffset=False)
plt.colorbar()
plt.tight_layout()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
#plt.savefig(image_dst+"/centralized_final_heatmap.png", dpi  = 150)
plt.show()

## KS test

In [ ]:
stats.ks_2samp(distributed_coverage.sort_values(by="id_location")['probability'], centralized_coverage.join(distributed_coverage.set_index('id_location'), on="id_location", how="right", rsuffix="_distr").sort_values(by="id_location")['probability'])

# Coverage probability 

In [ ]:


plt.figure(figsize=(10,7))

sns.histplot(data=centralized_coverage, x="probability", color = "g", alpha = 0.6)
plt.xlabel("Coverage probability")
plt.ylabel("")
savefig("cov_dist_scenario1.png", dpi  = 100)
